In [ ]:
"""Imports"""

import os
from ansys.edb.database import Database
from ansys.edb.layer import StackupLayer, LayerType
from ansys.edb.layout import Cell, CellType
from ansys.edb.net import Net
from ansys.edb.primitive import Circle, Text, Rectangle, RectangleRepresentationType

In [ ]:
"""Configs"""

# RPC_SERVER = None if separate server is already running, Root dir of server executable otherwise.
RPC_SERVER = None
ROOT = os.getcwd()
EDB_FILE = os.path.join(ROOT, "example_Define_and_assign_bondwire_profiles.aedb")

In [ ]:
"""Launch a session. Make sure old one has been disconnected prior to starting a new one, if any"""
session = launch_session(RPC_SERVER, 50051)

In [ ]:
"""Basic setup of empty EDB."""
db = Database.create(EDB_FILE)
cell = Cell.create(db, CellType.CIRCUIT_CELL, "EMDesign1")
layout = cell.layout

"""Nets for the different cells and create the primitives"""
net = Net.create(layout, 'GND')

In [ ]:
"""Setup Layers for cell"""
# Layers in layout's layer collection
layer_gnd = StackupLayer.create('Ground', LayerType.SIGNAL_LAYER, 0.0001, 0, "copper")
layer_1 = StackupLayer.create('NewLayer', LayerType.SIGNAL_LAYER, 0.0001, 0.0001, "copper")
lc = layout.layer_collection
lc.add_layers([layer_gnd, layer_1])

# Retrieve layers collection and layers
layout.layer_collection = lc
layer_gnd = lc.find_by_name('Ground')
layer_1 = lc.find_by_name('NewLayer')

In [ ]:
"""Create Primitive objects in ground layer"""
um = lambda val : val * 1e-6

text = Text.create(layout, layer_gnd, um(0.0), um(0.0), "my Text")

rect = Rectangle.create(layout,
                        layer_gnd,
                        net,
                        RectangleRepresentationType.LOWER_LEFT_UPPER_RIGHT,
                        um(10), um(10), um(20), um(20), 0.0, 0.0
                        )

rect_2 = Rectangle.create(layout,
                          layer_gnd,
                          net,
                          RectangleRepresentationType.LOWER_LEFT_UPPER_RIGHT,
                          um(0.0), um(0.0), um(30), um(30), 0.0, 0.0
                          )

circle = Circle.create(layout, layer_gnd, net, um(0.0), um(0.0), um(34.0))

# Add primitives to a list
primitives_created = [text, rect, rect_2, circle]

In [ ]:
"""Move Primitive objects to NewLayer"""
for primitive in primitives_created:
    primitive.layer = layer_1

In [ ]:
"""Save database if you wish."""
db.save()

In [ ]:
"""Don't forget to disconnect it!"""
session.disconnect()